# Problem 5.3  

**Integrated Energy Grids**

**Problem 5.3**

_Note: This is equivalent to Problem 2.2 which we solved using linopy, but here we will use PyPSA._

**Consider the following economic dispatch problem:**

**• we have three generators: solar, wind and gas**

**• solar and wind have no marginal costs, and gas has fuel costs of 60 EUR/MWh.**

**• we need to cover a electricity demand of 13.2 MWh**

**• the installed capacities are 15 MW, 20 MW and 20 MW for wind, solar, and gas, respectively**

**• assume the capacity factor for solar is 0.17 and for wind 0.33.**

**a) Use pypsa to find the optimal solution as well and obtain the electricity price (Lagrange multiplier for the energy balance constraint)**



:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install numpy pypsa
```
:::

In [13]:
import numpy as np
import pypsa

We start by creating the network object and adding the bus.

In [14]:
network = pypsa.Network()

In [15]:
network.add("Bus", "my bus") 
network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
my bus,1.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


We add the generators

In [16]:
network.add("Generator", 
            "solar", 
            bus="my bus", 
            p_nom=15, 
            p_max_pu=0.17,
            marginal_cost=0) 
    
network.add("Generator", 
            "wind", 
            bus="my bus", 
            p_nom=20, 
            p_max_pu=0.33,
            marginal_cost=0) 

network.add("Generator", 
            "gas", 
            bus="my bus", 
            p_nom=20, 
            marginal_cost=60) #EUR/MWh_elec
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
solar,my bus,PQ,,15.0,0.0,False,0.0,inf,0.0,0.17,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
wind,my bus,PQ,,20.0,0.0,False,0.0,inf,0.0,0.33,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gas,my bus,PQ,,20.0,0.0,False,0.0,inf,0.0,1.00,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


We add the load

In [17]:
network.add("Load", "electricity demand", bus="my bus", p_set=13.2)
network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
electricity demand,my bus,,,13.2,0.0,-1.0,True


Calculate the optimal economic dispatch

In [18]:
network.optimize()

Index(['my bus'], dtype='object', name='Bus')
Index(['my bus'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 3 primals, 7 duals
Objective: 2.43e+02
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


('ok', 'optimal')

Now we can show the optimal dispatch and the electricity price (marginal cost at the bus)

In [19]:
network.generators_t.p

Generator,solar,wind,gas
snapshot,,,
now,2.55,6.6,4.05


In [20]:
network.buses_t.marginal_price

Bus,my bus
snapshot,
now,60.0
